In [1]:
import requests
import random
import json

In [2]:
base_url = "http://localhost:8081"

In [3]:
resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"

In [4]:
resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [5]:
swaggerJson = resp.json()
apiEndpoints = swaggerJson["paths"].keys()
assert len(apiEndpoints)==45

In [6]:
inviteUrls = [f for f in swaggerJson["paths"].keys() if "/invite" in f]
instituteInvite = [u for u in inviteUrls if "/Institute" in u][0]
teacherInvite = [v for v in inviteUrls if "/Teacher" in v][0]
studentInvite = [w for w in inviteUrls if "/Student" in w][0]
print(instituteInvite,teacherInvite,studentInvite)

/api/v1/Institute/invite /api/v1/Teacher/invite /api/v1/Student/invite


In [7]:
def entityForTeacher(userId):
    return  {"personalDetails":{
         "userName":userId,
         "dob":"2020-12-12",
     },
      "IdentityDetails":{"type":"id","value":userId}, 
      "ContactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
                       "address":{"street":"abc","state":"Karnataka","pincode":"560001"}}
     }

In [36]:
def entityForTeacherWithExperiance(userId,instituteId):
    return  {"personalDetails":{
         "userName":userId,
         "dob":"2020-12-12",
     },
      "IdentityDetails":{"type":"id","value":userId}, 
      "ContactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
                       "address":{"street":"abc","state":"Karnataka","pincode":"560001"}},
      "academicQualifications":[{"institute":"%s_1"%instituteId}],
      "experience":[{"institute":"%s"%instituteId}]
     }

In [9]:
def entityForStudent(userId,instituteId):
    return  {
        "identityDetails":{
            "fullName": "Student %s"%userId, 
            "gender": "Male"
        },
        "ContactDetails":{
            "mobile":"%s"%userId,
            "email":"%s@example.com"%userId,
            "address":{
                "street":"abc",
                "state":"Karnataka",
                "pincode":"560001"}},
        "educationDetails":[
            {
                "institute":"%s"%instituteId,
                "program":"Engineering",
                "graduationYear":"2017",
                "marks":"95"
            }
        ]
    }

In [10]:
def entityForInstitute(userId):
    return  {
        "instituteName": "%s"%userId,
        "ContactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
               "address":{"street":"abcde","state":"Karnataka","pincode":"560001"}},
        "category": "Primary",
        "schoolType": "Co-ed"
    }

In [11]:
def getEntityPayload(userId,password):
    return {
      "client_id": "registry-frontend",
      "username": "%s"%userId,
      "password": "%s"%password,
      "grant_type": "password"
    }


In [12]:
defaultPassowrd = "abcd@123"

In [13]:
entityForStudent(2345,"4567899")

{'identityDetails': {'fullName': 'Student 2345', 'gender': 'Male'},
 'ContactDetails': {'mobile': '2345',
  'email': '2345@example.com',
  'address': {'street': 'abc', 'state': 'Karnataka', 'pincode': '560001'}},
 'educationDetails': [{'institute': '4567899',
   'program': 'Engineering',
   'graduationYear': '2017',
   'marks': '95'}]}

In [37]:
entityForTeacherWithExperiance(2345,"4567899")

{'personalDetails': {'userName': 2345, 'dob': '2020-12-12'},
 'IdentityDetails': {'type': 'id', 'value': 2345},
 'ContactDetails': {'mobile': '2345',
  'email': '2345@example.com',
  'address': {'street': 'abc', 'state': 'Karnataka', 'pincode': '560001'}},
 'academicQualifications': [{'institute': '4567899_1'}],
 'experience': [{'institute': '4567899'}]}

In [15]:
entityForTeacher(2345)

{'personalDetails': {'userName': 2345, 'dob': '2020-12-12'},
 'IdentityDetails': {'type': 'id', 'value': 2345},
 'ContactDetails': {'mobile': '2345',
  'email': '2345@example.com',
  'address': {'street': 'abc', 'state': 'Karnataka', 'pincode': '560001'}}}

# invite

### Institute

In [16]:
instituteId_1 =str(random.randint(1e10,1e11))
instituteId_1

'97054657802'

In [17]:
instituteId_2 =str(random.randint(1e10,1e11))
instituteId_2

'17022784195'

In [20]:
resp = requests.post("%s%s"%(base_url, instituteInvite), json=entityForInstitute(instituteId_1)
)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Institute"==entity_name
instituteOsId = resp.json()["result"][entity_name]["osid"]
instituteOsId

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1646885432520,"params":{"resmsgid":"","msgid":"f4526673-beb9-4299-81cb-05be674c6a50","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-4f0a9b74-6d29-41b3-8b5c-ceab36ccf39f"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1646885432520, 'params': {'resmsgid': '', 'msgid': 'f4526673-beb9-4299-81cb-05be674c6a50', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Institute': {'osid': '1-4f0a9b74-6d29-41b3-8b5c-ceab36ccf39f'}}}


'1-4f0a9b74-6d29-41b3-8b5c-ceab36ccf39f'

### Institute Token

In [43]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(instituteId_1,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
institutionToken = response.json()["access_token"]
institutionToken

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZcGNvVHQ4bTBveXZkbjY4dU9icE0zX1REZWwzbXFhWXA5d19hX19PY3g0In0.eyJleHAiOjE2NDY4ODgxNzIsImlhdCI6MTY0Njg4NzU3MiwianRpIjoiNGI3Yzg2OTctZDdjMy00OGE3LTliMDItZDNlZWFmMTQyZjAwIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiJiY2ZlODY4Mi1kODU4LTQ5NjktYTdiMy0yYTNmY2U0OGYzNzYiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJiZGVmNWM1Yy01NjQxLTQ4YTItYWVmYi0xNTc4YTViODAyNDUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImVkdS1hZG1pbiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidml

### Get Same Institute

In [22]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%institutionToken, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%(entity_name, instituteOsId), headers=headers)
print(response.status_code)
obj = response.json()
response.status_code, response.json()

200


(200,
 {'osUpdatedAt': '2022-03-10T04:11:42.366299Z',
  'ContactDetails': {'osid': '1-1c9be2cb-beb3-4436-afde-108701aee693',
   'osUpdatedAt': '2022-03-10T04:11:42.366299Z',
   'osCreatedAt': '2022-03-10T04:10:45.224024Z',
   'address': {'osid': '1-35489c23-fb2c-41cc-a93c-32ee1b3b9ccc',
    'pincode': '560001',
    'osUpdatedAt': '2022-03-10T04:11:42.366299Z',
    'osCreatedAt': '2022-03-10T04:10:45.224024Z',
    'osUpdatedBy': '',
    'street': 'abcde',
    'osCreatedBy': '',
    'state': 'Karnataka'},
   'osUpdatedBy': '',
   'osCreatedBy': '',
   'mobile': '97054657802',
   'email': '97054657802@example.com'},
  'osCreatedAt': '2022-03-10T04:10:45.224024Z',
  'osUpdatedBy': '',
  'schoolType': 'Co-ed',
  'osCreatedBy': '',
  'osid': '1-4f0a9b74-6d29-41b3-8b5c-ceab36ccf39f',
  'category': 'Primary',
  'osOwner': ['bcfe8682-d858-4969-a7b3-2a3fce48f376'],
  'InstituteAffiliation': [{'osUpdatedAt': '2022-03-10T04:11:42.366299Z',
    'osUpdatedBy': '',
    'entityName': 'Institute',
    

## Invite Teacher 1

In [23]:
teacherId_1 =str(random.randint(1e10,1e11))
teacherId_1

'51930068146'

In [24]:
teacherId_2 =str(random.randint(1e10,1e11))
teacherId_2

'30943612100'

In [44]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institutionToken,
}
resp = requests.post("%s%s"%(base_url, teacherInvite), json=entityForTeacherWithExperiance(teacherId_1,instituteId_1),headers= headers)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Teacher"==entity_name
teacherOsId = resp.json()["result"][entity_name]["osid"]

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1646887576945,"params":{"resmsgid":"","msgid":"6ac7d5dc-0405-405f-a0bb-a3b7aeb99458","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-55e1fc19-1c0b-4f02-8aa6-78220c838414"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1646887576945, 'params': {'resmsgid': '', 'msgid': '6ac7d5dc-0405-405f-a0bb-a3b7aeb99458', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Teacher': {'osid': '1-55e1fc19-1c0b-4f02-8aa6-78220c838414'}}}


## Invite Teacher 2

In [45]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institutionToken,
}
resp = requests.post("%s%s"%(base_url, teacherInvite), json=entityForTeacherWithExperiance(teacherId_2,instituteId_2),headers= headers)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Teacher"==entity_name
teacherOsId = resp.json()["result"][entity_name]["osid"]
teacherOsId

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1646887671095,"params":{"resmsgid":"","msgid":"7b6576fc-1d15-407d-a35a-aedd1f38e41c","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-67a0b0e1-c8d2-43d3-99ba-daef69613001"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1646887671095, 'params': {'resmsgid': '', 'msgid': '7b6576fc-1d15-407d-a35a-aedd1f38e41c', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Teacher': {'osid': '1-67a0b0e1-c8d2-43d3-99ba-daef69613001'}}}


'1-67a0b0e1-c8d2-43d3-99ba-daef69613001'

### Teacher Token

In [62]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(teacherId_1,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
teacherToken = response.json()["access_token"]
teacherToken

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZcGNvVHQ4bTBveXZkbjY4dU9icE0zX1REZWwzbXFhWXA5d19hX19PY3g0In0.eyJleHAiOjE2NDY4ODkzMzUsImlhdCI6MTY0Njg4ODczNSwianRpIjoiZjc1YzFjOGEtMDI0OC00ZjA2LWE5MTgtNGFkZjViMmQ5MWRhIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI1NjBlYjFkZC03OGIzLTQyMjItOGNmYy0xOTNlYTk3NDcwN2IiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiIzMzg4NjI2Mi1mODFhLTQwYzgtYjIxYS01OWFjYmFkNjkzZjkiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwiZWR1LXN0YWZmIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidml

### Teacher 2 Token

In [56]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(teacherId_2,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
teacher2Token = response.json()["access_token"]
teacher2Token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZcGNvVHQ4bTBveXZkbjY4dU9icE0zX1REZWwzbXFhWXA5d19hX19PY3g0In0.eyJleHAiOjE2NDY4ODkxNjgsImlhdCI6MTY0Njg4ODU2OCwianRpIjoiOTVlODg5MmYtOTI3Mi00ZDkwLTk0YTMtM2NmMTRlMzdiZDI1IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIxYTU4Yjc4Yi0zNTY4LTQ2MDYtODcyMy05YTQ1YmFhNGQ0YjEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJjMGQ4YmQxNS0wYTIwLTQ1ZmYtYTM3NC0wZjA1MmJhYWZjYzYiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwiZWR1LXN0YWZmIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidml

## invite a student

In [47]:
studentId =str(random.randint(1e10,1e11))
studentId

'64441813084'

In [50]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}
data = entityForStudent(studentId,instituteId_1)
resp = requests.post("%s%s"%(base_url, studentInvite), json=data,headers=headers)
assert resp.status_code == 200
print(resp.json())
studentOsId = resp.json()["result"]["Student"]["osid"]
studentOsId

{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1646887722729, 'params': {'resmsgid': '', 'msgid': '79d14e49-b962-47e9-9f39-db03eb6d72eb', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Student': {'osid': '1-abef0029-dacf-444e-9e58-857cc63e263d'}}}


'1-abef0029-dacf-444e-9e58-857cc63e263d'

In [59]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(studentId,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
studentToken = response.json()["access_token"]
studentToken

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZcGNvVHQ4bTBveXZkbjY4dU9icE0zX1REZWwzbXFhWXA5d19hX19PY3g0In0.eyJleHAiOjE2NDY4ODkzMTIsImlhdCI6MTY0Njg4ODcxMiwianRpIjoiN2U5MDFlMTgtYjkwNy00NWRhLTlkNWMtZDFlY2RkYmYxNDY5IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIwNzE4NGRkNi04ZjI4LTQ1Y2QtOTYxZS0wYzE2MTk5MWRlMjUiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJjOGRjNDZhNy1iYTc5LTQ4NzgtOTEzOS1kYWU0ZWRjNGVhNDMiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

### Get Student

In [65]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%studentToken, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%("Student", studentOsId), headers=headers)
print(response.status_code)
response.json()

200


{'ContactDetails': {'osid': '1-1ac659fb-9b52-4591-b142-f92a02643dc4',
  'address': {'osid': '1-8121a6f7-a8f7-454f-be6b-55d037d63778',
   'pincode': '560001',
   'street': 'abc',
   'state': 'Karnataka'},
  'mobile': '64441813084',
  'email': '64441813084@example.com'},
 'educationDetails': [{'institute': '17022784195',
   'graduationYear': '2017',
   'marks': '95',
   'program': 'Engineering',
   'osid': '1-def41c4c-b87f-4cb0-8e07-a567199cf42d'}],
 'osid': '1-abef0029-dacf-444e-9e58-857cc63e263d',
 'osOwner': ['07184dd6-8f28-45cd-961e-0c161991de25'],
 'identityDetails': {'osid': '1-e9a78d1d-d2f8-4e23-aa63-65973d25c56e',
  'gender': 'Male',
  'fullName': 'Student 64441813084'},
 'schoolId': [{'_osState': 'DRAFT',
   'entityName': 'Student',
   'name': 'schoolId',
   'entityId': '1-abef0029-dacf-444e-9e58-857cc63e263d',
   'osid': '1-b961aa6c-81ce-490a-a4e0-0c5301885a3a',
   'propertyData': '{"name":"Student 64441813084","institute":"333777779"}'},
  {'_osState': 'ATTESTATION_REQUESTED',

## Teacher 1 should get claim

In [63]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher/claims', headers=headers)
print(response.json())
claim_id = response.json()[-1]["id"]
response.json()

[{'id': 'fe04a3f5-2180-4f52-b7ab-343638a0f990', 'entity': 'Student', 'entityId': '1-abef0029-dacf-444e-9e58-857cc63e263d', 'propertyURI': '', 'createdAt': '2022-03-10T04:49:39.141+00:00', 'attestedOn': None, 'status': 'OPEN', 'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('97054657802'))", 'attestorEntity': 'Teacher', 'requestorName': None, 'propertyData': '{"name":"Student 64441813084","institute":"97054657802"}', 'attestationId': '1-f71ef510-5723-414a-b97e-abdb7de8c551', 'attestationName': 'schoolId', 'closed': False}]


[{'id': 'fe04a3f5-2180-4f52-b7ab-343638a0f990',
  'entity': 'Student',
  'entityId': '1-abef0029-dacf-444e-9e58-857cc63e263d',
  'propertyURI': '',
  'createdAt': '2022-03-10T04:49:39.141+00:00',
  'attestedOn': None,
  'status': 'OPEN',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('97054657802'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,
  'propertyData': '{"name":"Student 64441813084","institute":"97054657802"}',
  'attestationId': '1-f71ef510-5723-414a-b97e-abdb7de8c551',
  'attestationName': 'schoolId',
  'closed': False}]

### Update Student

In [64]:
headers = {
    'content-type': 'application/json',
    'authorization': 'Bearer %s'%studentToken,
}
data = entityForStudent(studentId,instituteId_2)
print(data)
response = requests.put('http://localhost:8081/api/v1/%s/%s'%("Student", studentOsId), headers=headers, json=data)
response.json()

{'identityDetails': {'fullName': 'Student 64441813084', 'gender': 'Male'}, 'ContactDetails': {'mobile': '64441813084', 'email': '64441813084@example.com', 'address': {'street': 'abc', 'state': 'Karnataka', 'pincode': '560001'}}, 'educationDetails': [{'institute': '17022784195', 'program': 'Engineering', 'graduationYear': '2017', 'marks': '95'}]}


{'id': 'sunbird-rc.registry.update',
 'ver': '1.0',
 'ets': 1646888744862,
 'params': {'resmsgid': '',
  'msgid': '9509b3be-765f-41d1-b1ca-7258256af076',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK'}

## Teacher 2 should get claim

In [66]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'bearer %s'%teacher2Token, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher/claims', headers=headers)
print(response.json())
claim_id = response.json()[-1]["id"]
response.json()

[{'id': '24621104-6bc5-4dd8-a26c-1e8020e639db', 'entity': 'Student', 'entityId': '1-abef0029-dacf-444e-9e58-857cc63e263d', 'propertyURI': '', 'createdAt': '2022-03-10T05:05:46.448+00:00', 'attestedOn': None, 'status': 'OPEN', 'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('17022784195'))", 'attestorEntity': 'Teacher', 'requestorName': None, 'propertyData': '{"name":"Student 64441813084","institute":"17022784195"}', 'attestationId': '1-735a44d8-7a7d-4265-9be6-454e28a1d424', 'attestationName': 'schoolId', 'closed': False}]


[{'id': '24621104-6bc5-4dd8-a26c-1e8020e639db',
  'entity': 'Student',
  'entityId': '1-abef0029-dacf-444e-9e58-857cc63e263d',
  'propertyURI': '',
  'createdAt': '2022-03-10T05:05:46.448+00:00',
  'attestedOn': None,
  'status': 'OPEN',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('17022784195'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,
  'propertyData': '{"name":"Student 64441813084","institute":"17022784195"}',
  'attestationId': '1-735a44d8-7a7d-4265-9be6-454e28a1d424',
  'attestationName': 'schoolId',
  'closed': False}]

## Teacher 2 approve's the claim

In [67]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%teacher2Token, 
}
data = {
    "action":"GRANT_CLAIM",
}
response = requests.post('http://localhost:8081/api/v1/Teacher/claims/%s/attest'%claim_id, headers=headers, json=data)
print(response.status_code)
response.json()

200


{'resmsgid': '',
 'msgid': '863d3dd3-8da6-4fb7-9db0-d8fcd97bb9fe',
 'err': '',
 'status': 'SUCCESSFUL',
 'errmsg': ''}

### Claim status should be Closed after Approval

In [70]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%teacher2Token, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher/claims', headers=headers)
print(response.status_code)
claim_id = response.json()[0]["id"]
assert response.json()[0]["status"]=='CLOSED'
response.json()

200


[{'id': '24621104-6bc5-4dd8-a26c-1e8020e639db',
  'entity': 'Student',
  'entityId': '1-abef0029-dacf-444e-9e58-857cc63e263d',
  'propertyURI': '',
  'createdAt': '2022-03-10T05:05:46.448+00:00',
  'attestedOn': '2022-03-10T05:06:27.351+00:00',
  'status': 'CLOSED',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('17022784195'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,
  'propertyData': '{"name":"Student 64441813084","institute":"17022784195"}',
  'attestationId': '1-735a44d8-7a7d-4265-9be6-454e28a1d424',
  'attestationName': 'schoolId',
  'closed': True}]